In [ ]:
from kafka import KafkaProducer
from wetterdienst.dwd.observations import DWDObservationData, DWDObservationParameterSet, DWDObservationResolution
from pymongo import MongoClient
import csv

client = MongoClient('mongodb://localhost:27017/')
stations = client.bda.stations.find({}, {'STATION_ID':1, "_id":0})


In [ ]:
producer = KafkaProducer()

def on_send_success(record_metadata):
    j = 0
    print(record_metadata.topic)
    print(record_metadata.partition)
    print(record_metadata.offset)

def on_send_error(excp):
    log.error('I am an errback', exc_info=excp)
    # handle exception

for station in stations:
    station_id = station['STATION_ID']

    observations = DWDObservationData(
    station_ids=[station_id],
    parameters=[DWDObservationParameterSet.TEMPERATURE_AIR],
    resolution=DWDObservationResolution.HOURLY,
    start_date="2020-07-01",
    end_date="2020-08-31",
    tidy_data=True,
    humanize_column_names=True,
    )

    print(type(DWDObservationParameterSet.TEMPERATURE_AIR))

    for df in observations.collect_data():
        df = df[df['ELEMENT'].str.match('TEMPERATURE_AIR_200')]
        raw = df.to_csv()
        lines = csv.reader(raw.splitlines()[1:])

        for i, line in enumerate(lines):
            producer.send('weather', value=bytearray(str(line).strip('[]'), encoding='utf-8'), key=bytearray(str(i), encoding='utf-8')).add_callback(on_send_success).add_errback(on_send_error)